In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

490

In [3]:
!mkdir pages

In [22]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            l = url.split('.html')[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{l}-{i}.html')

100%|███████████████████████████████████████| 490/490 [00:00<00:00, 4504.92it/s]


In [24]:
urls = sorted(list(set(urls)))

In [25]:
len(urls)

20806

In [26]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find_all('div')
    div = [str(div_) for div_ in div if 'post_message' in div_.get('id', '')]
    return div

get_url(urls[0])

['<div id="post_message_113932">\nwant to know a secret and easy way to buy and/or sell bitcoin?<br/>\n<br/>\nBe one of our firsts to participate in our new version of Telegram. ItSybit, your 24/7 trading partner on our app, Telegram.<br/>\n<br/>\nQUICK, HURRY, GET A SPOT NOW BY CLICKING ON THE LINK BELOW<br/>\n<br/>\nContact Sammie Cheston for questions and more information.<br/>\n<br/>\nGET STARTED NOW!!!<br/>\n\x95Download the app \x93Telegram<br/>\n\x95Click or copy the link here: s://t.me/itsybit_bot?referral_code=2khgv3d1v<br/>\n\x95Enter this referral code to participate: 2khgv3d1v\n</div>',
 '<div id="post_message_113934">\nwant to know a secret and easy way to buy and/or sell bitcoin?<br/>\n<br/>\nBe one of our firsts to participate in our new version of Telegram. ItSybit, your 24/7 trading partner on our app, Telegram.<br/>\n<br/>\nQUICK, HURRY, GET A SPOT NOW BY CLICKING ON THE LINK BELOW<br/>\n<br/>\nContact Sammie Cheston for questions and more information.<br/>\n<br/>\nGE

In [28]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|█████████████████████████████████████████| 417/417 [43:51<00:00,  6.31s/it]


In [29]:
files = glob('threads/*.json')
len(files)

490

In [30]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████████| 490/490 [00:06<00:00, 72.70it/s]


In [31]:
files = glob('pages/*.json')
len(files)

417

In [32]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████████| 417/417 [00:44<00:00,  9.36it/s]


In [33]:
texts = list(set(texts))
len(texts)

231202

In [34]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|███████████████████████████████| 231202/231202 [00:00<00:00, 569394.98it/s]


In [36]:
# !zip -r threads.zip threads
# !zip -r pages.zip pages
# !zip -r forums.zip forums